In [1]:
from torchvision import transforms, utils
import numpy as np

import os

from PIL import Image
from pathlib import Path

from tqdm.notebook import tqdm

In [2]:
### parameters
src_dir = "/raid/elhamod/CuratedDorsalBackgroundRemovedSplitFilteredSubspecies"
target_dir = "/raid/elhamod/CuratedDorsalBackgroundRemovedSplitFilteredSubspecies_resizedpadded_128pixels" 

In [3]:
#### MakeSquared function

IMAGENET_MEAN_COLOR=np.asarray([ 0.485, 0.456, 0.406 ])

def MakeSquared(img, imageDimension=128, mean=None):
    img_H = img.size[0]
    img_W = img.size[1]

    # Resize and pad
    smaller_dimension = 0 if img_H < img_W else 1
    larger_dimension = 1 if img_H < img_W else 0
    if (imageDimension != img_H or imageDimension != img_W):
        new_smaller_dimension = int(imageDimension * img.size[smaller_dimension] / img.size[larger_dimension])
        if smaller_dimension == 1:
            img = transforms.functional.resize(img, (new_smaller_dimension, imageDimension))
        else:
            img = transforms.functional.resize(img, (imageDimension, new_smaller_dimension))

        diff = imageDimension - new_smaller_dimension
        pad_1 = int(diff/2)
        pad_2 = diff - pad_1
        
        # if mean=None is passed, use the color of the top left pixel.
        if mean is None:
            mean = img.load()[0, 0]
        
        fill = mean

        if smaller_dimension == 0:
            img = transforms.functional.pad(img, (pad_1, 0, pad_2, 0), padding_mode='constant', fill = fill)
        else:
            img = transforms.functional.pad(img, (0, pad_1, 0, pad_2), padding_mode='constant', fill = fill)

    return img

We will iterate through the images in a folder, make them squared, and then save them in target folder

In [4]:
for root, dirs, files in tqdm(os.walk(src_dir, topdown = False)):
    for name in files:
        if (name.endswith(".png")):
            filename = os.path.join(root, name)
            orig_img = Image.open(filename)
            target_img = MakeSquared(orig_img)
            target_path = os.path.join(target_dir, os.path.relpath(root, src_dir))
            if not os.path.exists(target_path):
                os.makedirs(target_path)
            target_img.save(os.path.join(target_path, name))
        